In [8]:
import pandas as pd
from haversine import haversine as hvs, Unit
import numpy as np
from collections import Counter
import math
from datetime import datetime, timedelta

# Procesar samples_with_nan

Crear una columna con un indice para los samples (1 sample es un recorrido completo)

In [11]:
# samples_with_nan = pd.read_csv('../data/samples_with_nan.csv', index_col=None)
samples_with_nan = pd.read_csv('../data/pcampo.csv', index_col=None)

# len(samples_with_nan['init_stop'].unique())   #83 paraderos  # 2560 samples con nan
#samples_with_nan['id_sample'] = samples_with_nan.apply(lambda row: int(row.name)//82) 
#samples_with_nan.apply(lambda row: int(row.name))#(//82) 
samples_with_nan['id_sample'] = samples_with_nan.index.map(lambda row: int(row)//83)
columna = samples_with_nan.pop('id_sample') # Extraemos la columna C y la guardamos en una variable
samples_with_nan.insert(0, 'id_sample', columna) # Insertamos la columna C en la posición 1
#samples_with_nan.to_csv('samples_with_nan.csv')
samples_with_nan.to_csv('samples_with_nan.csv', index=False)
samples_with_nan

,id_sample,init_stop,name,end_stop,name_end_stop,linkref,date_time_init,date_time_end,time_travel,init_idp,end_idp,veh,lap,date
0,0,0,inicio,36,primer stop,0:36,2023-07-06 08:56:58,2023-07-06 08:58:25,87.0,8798.0,11645.0,SJ01,2.0,2023-07-06
1,0,36,primer stop,37,segundo stop,36:37,2023-07-06 08:58:25,2023-07-06 08:59:25,60.0,11645.0,13599.0,SJ01,2.0,2023-07-06
2,0,37,segundo stop,38,Ccollana,37:38,2023-07-06 08:59:25,2023-07-06 09:00:29,64.0,13599.0,15682.0,SJ01,2.0,2023-07-06
3,0,38,Ccollana,39,Puente Huaccoto,38:39,2023-07-06 09:00:29,2023-07-06 09:01:15,46.0,15682.0,17180.0,SJ01,2.0,2023-07-06
4,0,39,Puente Huaccoto,40,Kayra,39:40,2023-07-06 09:01:15,2023-07-06 09:02:05,50.0,17180.0,18858.0,SJ01,2.0,2023-07-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4892,58,113,Primero,114,Segundo,113:114,2023-07-06 16:13:01,2023-07-06 16:13:13,12.0,34996.0,35485.0,SJ43,5.0,2023-07-06
4893,58,114,Segundo,115,Reservorio,114:115,2023-07-06 16:13:13,2023-07-06 16:13:49,36.0,35485.0,17660.0,SJ43,5.0,2023-07-06
4894,58,115,Reservorio,116,Cuarto,115:116,2023-07-06 16:13:49,2023-07-06 16:14:01,12.0,17660.0,18089.0,SJ43,5.0,2023-07-06
4895,58,116,Cuarto,117,Quinto,116:117,2023-07-06 16:14:01,2023-07-06 16:14:13,12.0,18089.0,18587.0,SJ43,5.0,2023-07-06


Si no se borró la columna 'Unnamed: 0', eliminarla y volver a guardar el dataframe en csv

In [10]:
#EJECUTAR CUANDO EXISTA LA COLUMNA UNNAMED 0
samples_with_nan = pd.read_csv('samples_with_nan.csv', index_col=None)
samples_with_nan.drop(columns=['Unnamed: 0'], inplace=True)
samples_with_nan.to_csv('samples_with_nan.csv', index=False)
samples_with_nan

KeyError: "['Unnamed: 0'] not found in axis"

### Filtro de samples de la columna time_travel para mostrar todos los menores a 12 segundos

In [12]:
# samples_with_nan = pd.read_csv('../data/samples_with_nan.csv', index_col=None)
samples_with_nan = pd.read_csv('../data/pcampo.csv', index_col=None)
#samples_with_nan.nlargest(50,columns='time_travel')
#samples_with_nan[samples_with_nan['time_travel']>=960]['id_sample'].unique #960 segundos equivale a 10 minutos
samples_with_nan[samples_with_nan['time_travel']<12]['id_sample'].unique().shape #960 segundos equivale a 10 minutos

(0,)

In [13]:
samples_with_nan.nlargest(30, 'time_travel')

,id_sample,init_stop,name,end_stop,name_end_stop,linkref,date_time_init,date_time_end,time_travel,init_idp,end_idp,veh,lap,date
581,0,0,inicio,36,primer stop,0:36,2023-07-06 11:22:46,2023-07-06 11:43:43,1257.0,13735.0,32536.0,SJ05,3.0,2023-07-06
1245,0,0,inicio,36,primer stop,0:36,2023-07-06 13:32:36,2023-07-06 13:45:28,772.0,21692.0,9980.0,SJ13,4.0,2023-07-06
3070,82,117,Quinto,118,Sexto,117:118,2023-07-06 12:55:08,2023-07-06 13:05:55,647.0,25606.0,8673.0,SJ27,3.0,2023-07-06
3874,56,91,Belén,92,Centro comercial Paraíso,91:92,2023-07-06 12:14:59,2023-07-06 12:22:15,436.0,21902.0,42498.0,SJ34,3.0,2023-07-06
3044,56,91,Belén,92,Centro comercial Paraíso,91:92,2023-07-06 12:31:39,2023-07-06 12:37:41,362.0,17152.0,28521.0,SJ27,3.0,2023-07-06
3293,56,91,Belén,92,Centro comercial Paraíso,91:92,2023-07-06 12:48:53,2023-07-06 12:54:48,355.0,13728.0,24740.0,SJ28,3.0,2023-07-06
58,58,93,Puente Santiago,94,Plaza Santiago,93:94,2023-07-06 09:52:32,2023-07-06 09:58:16,344.0,17070.0,28443.0,SJ01,2.0,2023-07-06
1411,0,0,inicio,36,primer stop,0:36,2023-07-06 13:44:34,2023-07-06 13:50:18,344.0,8139.0,19496.0,SJ15,8.0,2023-07-06
1079,0,0,inicio,36,primer stop,0:36,2023-07-06 13:37:39,2023-07-06 13:43:02,323.0,30939.0,4892.0,SJ12,4.0,2023-07-06
4537,55,90,Puente Grau,91,Belén,90:91,2023-07-06 12:46:44,2023-07-06 12:51:57,313.0,9978.0,19470.0,SJ41,2.0,2023-07-06


In [17]:
samples_with_nan.nsmallest(30, 'time_travel')

,id_sample,init_stop,name,end_stop,name_end_stop,linkref,date_time_init,date_time_end,time_travel,init_idp,end_idp,veh,lap,date
10,10,45,Tercero,46,Medio,45:46,2023-07-06 09:04:39,2023-07-06 09:04:51,12.0,23484.0,23879.0,SJ01,2.0,2023-07-06
20,20,55,Tingo,56,Teléfono,55:56,2023-07-06 09:15:56,2023-07-06 09:16:08,12.0,10924.0,11323.0,SJ01,2.0,2023-07-06
29,29,64,Mercadillo,65,Santa Rosa,64:65,2023-07-06 09:21:23,2023-07-06 09:21:35,12.0,22296.0,22412.0,SJ01,2.0,2023-07-06
32,32,67,Séptimo,68,Sexto,67:68,2023-07-06 09:23:45,2023-07-06 09:23:57,12.0,26951.0,27066.0,SJ01,2.0,2023-07-06
77,77,112,Subida,113,Primero,112:113,2023-07-06 10:11:05,2023-07-06 10:11:17,12.0,19280.0,19777.0,SJ01,2.0,2023-07-06
81,81,116,Cuarto,117,Quinto,116:117,2023-07-06 10:12:50,2023-07-06 10:13:02,12.0,23025.0,23705.0,SJ01,2.0,2023-07-06
101,18,53,San Martín,54,Maestro,53:54,2023-07-06 12:46:19,2023-07-06 12:46:31,12.0,9237.0,9564.0,SJ01,3.0,2023-07-06
113,30,65,Santa Rosa,66,Camionero,65:66,2023-07-06 12:53:46,2023-07-06 12:53:58,12.0,22848.0,23365.0,SJ01,3.0,2023-07-06
130,47,82,Amauta,83,Servicentro,82:83,2023-07-06 13:12:42,2023-07-06 13:12:54,12.0,20889.0,21298.0,SJ01,3.0,2023-07-06
160,77,112,Subida,113,Primero,112:113,2023-07-06 13:45:49,2023-07-06 13:46:01,12.0,10867.0,11256.0,SJ01,3.0,2023-07-06


In [18]:
samples_with_nan['time_travel'].isna().sum()

712

### Filtro de samples de la columna time_travel para moestrar todos los mayores a 960 segundos(10 minutos)

Mostrar el indice de los samples con time_travel mayor a 10 minutos

In [15]:
# samples_with_nan = pd.read_csv('../data/samples_with_nan.csv', index_col=None)
samples_with_nan = pd.read_csv('../data/pcampo.csv', index_col=None)

#samples_with_nan.nlargest(50,columns='time_travel')
#samples_with_nan[samples_with_nan['time_travel']>=960]['id_sample'].unique #960 segundos equivale a 10 minutos
samples_with_nan[samples_with_nan['time_travel']>=960]['id_sample'].unique() #960 segundos equivale a 10 minutos

array([0], dtype=int64)

Filtrar para eliminar y buscar samples manualmente con errores en el csv

In [16]:
#1182 DELETED SAMPLE 
#La ruta 0:36 tiene tiempos largos que fueron considerados.
# samples_with_nan = pd.read_csv('../data/samples_with_nan.csv', index_col=None)
samples_with_nan = pd.read_csv('../data/pcampo.csv', index_col=None)
samples_with_nan[samples_with_nan['id_sample']==1132].nlargest(20,'time_travel')

,id_sample,init_stop,name,end_stop,name_end_stop,linkref,date_time_init,date_time_end,time_travel,init_idp,end_idp,veh,lap,date


Crear un dataframe nuevo con datos limpiados manualmente (tiempos de time_travel legitimos)

In [125]:
# samples_with_nan_part1 = pd.read_csv('../data/samples_with_nan-0-816.csv', index_col=None)
# samples_with_nan_part2 = pd.read_csv('../data/samples_with_nan-817-2560.csv', index_col=None)

samples_with_nan_part1 = samples_with_nan_part1[samples_with_nan_part1['id_sample']<=816]
samples_with_nan_part2 = samples_with_nan_part2[samples_with_nan_part2['id_sample']>816]
unido = pd.concat([samples_with_nan_part1,samples_with_nan_part2], axis=0)

#Guardar el dataframe generado
unido.to_csv('../data/samples_with_nan_fixed.csv', index=False)
#Mostrar los 20 primeros datos mayores con el time_travel
unido.nlargest(20,'time_travel')

,id_sample,init_stop,end_stop,linkref,date_time_init,date_time_end,time_travel,init_idp,end_idp,veh,lap,date
51543,621,0,36,0:36,2023-03-06 09:57:29,2023-03-06 10:30:40,1991.0,14026875.0,14032850.0,SJ27,2.0,2023-03-06
10541,127,0,36,0:36,2023-03-01 14:04:39,2023-03-01 14:37:49,1990.0,13271047.0,13277666.0,SJ43,3.0,2023-03-01
67811,817,0,36,0:36,2023-03-08 13:13:37,2023-03-08 13:43:28,1791.0,14400162.0,14405374.0,SJ42,4.0,2023-03-08
15355,185,0,36,0:36,2023-03-02 09:16:27,2023-03-02 09:44:59,1712.0,13398974.0,13404743.0,SJ20,2.0,2023-03-02
156040,1881,0,36,0:36,2023-03-19 06:57:27,2023-03-19 07:25:01,1654.0,16172641.0,16175144.0,SJ15,1.0,2023-03-19
830,10,0,36,0:36,2023-03-01 14:02:10,2023-03-01 14:29:17,1627.0,13270563.0,13275804.0,SJ04,4.0,2023-03-01
108066,1303,0,36,0:36,2023-03-13 09:48:29,2023-03-13 10:14:56,1587.0,15148380.0,15153134.0,SJ41,2.0,2023-03-13
17845,215,0,36,0:36,2023-03-02 14:18:57,2023-03-02 14:44:37,1540.0,13461235.0,13465954.0,SJ28,4.0,2023-03-02
81091,977,0,36,0:36,2023-03-10 09:13:27,2023-03-10 09:38:50,1523.0,14704988.0,14710130.0,SJ19,2.0,2023-03-10
60839,733,0,36,0:36,2023-03-07 09:20:49,2023-03-07 09:45:42,1493.0,14192746.0,14197128.0,SJ41,2.0,2023-03-07


In [33]:
samples_nan_backup = pd.read_csv('../data/samples_nan_backup.csv', index_col=None)
samples_nan_backup[(samples_nan_backup['linkref']=='0:36') & (samples_nan_backup['time_travel']>960)].shape

(39, 12)

In [35]:
samples_nan_backup[samples_nan_backup['time_travel']>960].shape

(86, 12)

In [36]:
86-39

47

In [12]:
samples_with_nan.nsmallest(30, 'time_travel')

,id_sample,init_stop,end_stop,linkref,date_time_init,date_time_end,time_travel,init_idp,end_idp,veh,lap,date
167873,2022,82,83,82:83,2023-03-20 06:13:43,2023-03-20 06:13:53,10.0,16269633.0,16269656.0,SJ37,1.0,2023-03-20
1014,12,53,54,53:54,2023-03-01 10:55:49,2023-03-01 10:56:00,11.0,13233012.0,13233054.0,SJ05,2.0,2023-03-01
32070,386,67,68,67:68,2023-03-04 14:10:54,2023-03-04 14:11:05,11.0,13820185.0,13820219.0,SJ05,4.0,2023-03-04
46344,558,65,66,65:66,2023-03-06 12:54:52,2023-03-06 12:55:03,11.0,14059032.0,14059079.0,SJ05,3.0,2023-03-06
53899,649,67,68,67:68,2023-03-06 09:52:49,2023-03-06 09:53:00,11.0,14026068.0,14026111.0,SJ41,2.0,2023-03-06
117409,1414,82,83,82:83,2023-03-14 07:02:29,2023-03-14 07:02:40,11.0,15288247.0,15288284.0,SJ41,1.0,2023-03-14
129433,1559,71,72,71:72,2023-03-16 09:18:19,2023-03-16 09:18:30,11.0,15669142.0,15669187.0,SJ05,2.0,2023-03-16
146127,1760,82,83,82:83,2023-03-17 05:25:43,2023-03-17 05:25:54,11.0,15816980.0,15816990.0,SJ37,1.0,2023-03-17
147149,1772,108,109,108:109,2023-03-17 13:14:12,2023-03-17 13:14:23,11.0,15904671.0,15904705.0,SJ41,3.0,2023-03-17
147154,1772,113,114,113:114,2023-03-17 13:17:33,2023-03-17 13:17:44,11.0,15905251.0,15905286.0,SJ41,3.0,2023-03-17


# Porcentaje de nan en time_travel

In [6]:
samples_with_nan = pd.read_csv('../data/samples_with_nan.csv', index_col=None)
#samples_with_nan.nsmallest(50,columns='time_travel')
samples_with_nan['time_travel'].isna().sum()/samples_with_nan['time_travel'].shape[0]

0.14896460843373494

## Filtrar por fecha 

In [7]:
samples_with_nan = pd.read_csv('../data/samples_with_nan.csv', index_col=None)
samples_with_nan['date_time_init'] = pd.to_datetime(samples_with_nan['date_time_init'])
samples_with_nan['date_time_end'] = pd.to_datetime(samples_with_nan['date_time_end'])

fecha_filtro = pd.to_datetime('2023-03-12').date()
df_filtrado = samples_with_nan[samples_with_nan['date_time_init'].dt.date == fecha_filtro]
df_filtrado.nsmallest(20, ['time_travel'])

,id_sample,init_stop,end_stop,linkref,date_time_init,date_time_end,time_travel,init_idp,end_idp,veh,lap,date
95540,1151,42,43,42:43,2023-03-12 11:55:46,2023-03-12 11:55:58,12.0,15047959.0,15047983.0,SJ04,1.0,2023-03-12
95541,1151,43,44,43:44,2023-03-12 11:55:58,2023-03-12 11:56:10,12.0,15047983.0,15048008.0,SJ04,1.0,2023-03-12
95553,1151,55,56,55:56,2023-03-12 12:10:40,2023-03-12 12:10:52,12.0,15049606.0,15049626.0,SJ04,1.0,2023-03-12
95581,1151,83,84,83:84,2023-03-12 12:30:36,2023-03-12 12:30:48,12.0,15052341.0,15052344.0,SJ04,1.0,2023-03-12
95604,1151,106,107,106:107,2023-03-12 12:57:31,2023-03-12 12:57:43,12.0,15055259.0,15055291.0,SJ04,1.0,2023-03-12
95606,1151,108,109,108:109,2023-03-12 12:57:59,2023-03-12 12:58:11,12.0,15055322.0,15055343.0,SJ04,1.0,2023-03-12
95611,1151,113,114,113:114,2023-03-12 13:01:13,2023-03-12 13:01:25,12.0,15055652.0,15055672.0,SJ04,1.0,2023-03-12
95613,1151,115,116,115:116,2023-03-12 13:02:27,2023-03-12 13:02:39,12.0,15055782.0,15055803.0,SJ04,1.0,2023-03-12
95614,1151,116,117,116:117,2023-03-12 13:02:39,2023-03-12 13:02:51,12.0,15055803.0,15055820.0,SJ04,1.0,2023-03-12
95626,1152,45,46,45:46,2023-03-12 06:28:34,2023-03-12 06:28:46,12.0,15013171.0,15013176.0,SJ05,1.0,2023-03-12


In [8]:
samples_with_nan = pd.read_csv('../data/samples_1v.csv', index_col=None)
samples_with_nan['date_time_init'] = pd.to_datetime(samples_with_nan['date_time_init'])
samples_with_nan['date_time_end'] = pd.to_datetime(samples_with_nan['date_time_end'])

fecha_filtro = pd.to_datetime('2023-03-12').date()
df_filtrado = samples_with_nan[samples_with_nan['date_time_init'].dt.date == fecha_filtro]
df_filtrado.nsmallest(20, ['time_travel'])

,number,init_stop,end_stop,linkref,date_time_init,date_time_end,time_travel,init_idp,end_idp,veh,lap,date
98150,44,79,80,79:80,2023-03-12 11:49:25,2023-03-12 08:53:30,-10555.0,15047288.0,15023026.0,SJ31,1.0,2023-03-12
98182,76,111,112,111:112,2023-03-12 12:19:34,2023-03-12 09:24:00,-10534.0,15051151.0,15027886.0,SJ31,1.0,2023-03-12
98176,70,105,106,105:106,2023-03-12 12:16:32,2023-03-12 09:21:06,-10526.0,15050280.0,15027629.0,SJ31,1.0,2023-03-12
98162,56,91,92,91:92,2023-03-12 12:02:40,2023-03-12 09:09:26,-10394.0,15048763.0,15026206.0,SJ31,1.0,2023-03-12
98158,52,87,88,87:88,2023-03-12 11:02:03,2023-03-12 09:03:47,-7096.0,NaN,NaN,NaN,NaN,NaN
98157,51,86,87,86:87,2023-03-12 11:56:29,2023-03-12 11:02:03,-3266.0,NaN,NaN,NaN,NaN,NaN
96607,78,113,114,113:114,2023-03-12 11:13:07,2023-03-12 11:13:15,8.0,NaN,NaN,NaN,NaN,NaN
96537,8,43,44,43:44,2023-03-12 10:08:41,2023-03-12 10:08:51,10.0,NaN,NaN,NaN,NaN,NaN
96856,78,113,114,113:114,2023-03-12 14:02:36,2023-03-12 14:02:46,10.0,NaN,NaN,NaN,NaN,NaN
96268,71,106,107,106:107,2023-03-12 11:51:49,2023-03-12 11:52:00,11.0,NaN,NaN,NaN,NaN,NaN


## Calcular promedio

In [47]:
dt_with_nan = pd.read_csv('../data/samples_with_nan_fixed.csv', sep=',')
dt_with_nan['date_time_init'] = pd.to_datetime(dt_with_nan['date_time_init'])
dt_with_nan['date_time_end'] = pd.to_datetime(dt_with_nan['date_time_end'])
dt_with_nan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212397 entries, 0 to 212396
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   id_sample       212397 non-null  int64         
 1   init_stop       212397 non-null  int64         
 2   end_stop        212397 non-null  int64         
 3   linkref         212397 non-null  object        
 4   date_time_init  180721 non-null  datetime64[ns]
 5   date_time_end   180721 non-null  datetime64[ns]
 6   time_travel     180721 non-null  float64       
 7   init_idp        180721 non-null  float64       
 8   end_idp         180721 non-null  float64       
 9   veh             180721 non-null  object        
 10  lap             180721 non-null  float64       
 11  date            180721 non-null  object        
dtypes: datetime64[ns](2), float64(4), int64(3), object(3)
memory usage: 19.4+ MB


In [48]:
dt_with_nan['time_travel'].isna().sum()/dt_with_nan.shape[0]

0.14913581641925264

In [49]:
id_samples = dt_with_nan['id_sample'].unique().tolist()
len(id_samples)

2559

In [50]:
means_dict = dt_with_nan.groupby('linkref', observed=False).agg({'time_travel':[np.mean]})[('time_travel', 'mean')].to_list()
len(means_dict)

83

In [51]:
from more_itertools import consecutive_groups

def intervals(numeros):
    intervals = []
    for group in consecutive_groups(numeros):
        group = list(group)
        if len(group) == 1:
            intervals.append(tuple(group))
        else:
            intervals.append(tuple(group))

    return intervals

In [53]:
list_df_listos = [] 
for _, df_sample in dt_with_nan.groupby('id_sample'):
    # identificar los intervalos con nans
    df_sample.reset_index(inplace=True)
    index_nans = df_sample.index[pd.isna(df_sample['time_travel'])]
    inter_nans = intervals(list(index_nans))
    df_sample.insert(loc=2, column='id_linkref', value=df_sample.index)

    for gap in inter_nans:
        # generar los time travel
        means = [means_dict[i] for i in gap]

        # si hay nans en el medio, ajustamos means
        if gap[0] != 0 and gap[-1] != 82:
            time_init = df_sample.loc[gap[0] - 1, 'date_time_end']
            time_end = df_sample.loc[gap[-1] + 1, 'date_time_init']
            dif_time =  time_end - time_init
            means = list(map(lambda e: dif_time.total_seconds() * (e / sum(means)), means))

        # colocamos means
        for i in range(len(gap)):
            df_sample.loc[gap[i], 'time_travel'] = round(means[i], 0)
            if 0 not in gap:
                df_sample.loc[gap[i], 'date_time_init'] = df_sample.loc[gap[i]-1, 'date_time_end']
                df_sample.loc[gap[i], 'date_time_end'] = df_sample.loc[gap[i], 'date_time_init'] + timedelta(seconds=round(means[i], 0))
            else:
                j = (i+1)*(-1)
                df_sample.loc[gap[j], 'date_time_end'] = df_sample.loc[gap[j]+1, 'date_time_init']
                df_sample.loc[gap[j], 'date_time_init'] = df_sample.loc[gap[j], 'date_time_end'] - timedelta(seconds=round(means[i], 0))
    list_df_listos.append(df_sample)


In [54]:
print(list_df_listos[2000].to_string())

     index  id_sample  id_linkref  init_stop  end_stop  linkref      date_time_init       date_time_end  time_travel    init_idp     end_idp   veh  lap        date
0   166000       2001           0          0        36     0:36 2023-03-20 12:23:38 2023-03-20 12:24:52         74.0  16340049.0  16340279.0  SJ28  3.0  2023-03-20
1   166001       2001           1         36        37    36:37 2023-03-20 12:24:52 2023-03-20 12:26:15         83.0  16340279.0  16340536.0  SJ28  3.0  2023-03-20
2   166002       2001           2         37        38    37:38 2023-03-20 12:26:15 2023-03-20 12:27:59        104.0  16340536.0  16340844.0  SJ28  3.0  2023-03-20
3   166003       2001           3         38        39    38:39 2023-03-20 12:27:59 2023-03-20 12:28:47         48.0  16340844.0  16340991.0  SJ28  3.0  2023-03-20
4   166004       2001           4         39        40    39:40 2023-03-20 12:28:47 2023-03-20 12:29:25         38.0  16340991.0  16341112.0  SJ28  3.0  2023-03-20
5   166005      

In [46]:
df_tmp = pd.concat(list_df_listos)
# df_tmp.to_csv('../data/samples_without_nan.csv')